# El commentator

In [4]:
import os
import functools
import pandas as pd
import tabula
import re
from dotenv import load_dotenv
from unidecode import unidecode

In [5]:
pd.options.display.max_colwidth = 500

In [ ]:
load_dotenv()

## Dataframe extraction for a single pdf file

In [12]:
year = int(os.getenv("YEAR"))
year

In [13]:
# Check if out/{year} directory exists and create it otherwise
if not os.path.exists(f"./out/{year}"):
  os.mkdir(f"./out/{year}")

In [4]:
pdf_path = f"./Bulletins/{year}/ABILIOU Anawen-Trimestre 1-2NDE 4-C-2840.pdf"
pdf_path2 = f"./Bulletins/{year}/AL BOUKHARI Zyneb-Trimestre 2-2NDE 3-N-2675.pdf"

df = tabula.read_pdf(
    pdf_path2,
     pages=1,
     stream=False,
     lattice=True,
     guess=True,
     )[0]

# Drop 'field-name' column & last NaN column 
df = df.drop([df.columns[0], df.columns[6]], axis=1) 

# Drop rows with no comment
df = df.dropna(subset=[df.columns[4]]) 


df.columns = ['eleve', 'classe', "min", "max", "commentaire"]

convert_dict = {
    'eleve': float,
    'classe': float,
    'min': float,
    'max': float,
    'commentaire': str
 }

# Pre-process columns 
df = df.apply(lambda x: x.replace(',','.', regex=True) if x.name != 'commentaire' else x, axis=1)
## Filter rows with multiple student grades values
df = df[~df['eleve'].str.contains('\r')]

df = df.astype(convert_dict)

# Remove returns in comments
df['commentaire'] = df['commentaire'].str.replace('\r',' ', regex=True)

df

,eleve,classe,min,max,commentaire
2,11.00,11.97,2.17,19.33,Ensemble convenable malgré un investissement inégal. Poursuivez vos efforts dans la régularité. sans vous décourager.
3,9.00,10.00,1.00,15.00,Un trimestre assez moyen même si l'attitude en cours et dans le travail est satisfaisante. Il faut accentuer les efforts et gagner en rigueur. tu es capable de mieux !
4,15.00,12.65,0.00,16.50,Un engagement positif dans le travail de groupe. Zyneb a su mettre ses qualités individuelles au service d'un projet commun.
5,13.87,13.80,8.62,17.38,Bilan moyen et sérieux. très encourageant. Poursuivez vos efforts.
6,10.86,13.27,8.50,17.14,Des résultats moyens. L'écrit est fragile. Cependant. de la volonté et une participation appréciée à l'oral. Avec un investissement personnel plus franc. vous devriez pouvoir progresser.
7,8.50,12.39,8.29,18.00,Un ensemble qui reste bien trop juste. L'écrit est souventmaladroit et l'attention en classe. fluctuante. Comme au 1er trimestre. je pense que Zyneb peut bien mieux.
8,10.44,10.78,2.97,18.65,Ensemble moyen. Il faut poursuivre vos efforts et approfondir le travail. Vous pouvez progresser.
9,11.41,10.18,2.57,17.06,Résultats moyens néanmoins l'attitude en cours s'est nettement améliorée. Continuez ainsi pour progresser.
10,10.00,11.28,0.00,17.50,Niveau trop moyen ; élève qui participe régulièrement. mais dont le travail manque de rigueur et l'attitude de concentration. Il faut rectifier ces manquements pour nous convaincre que la spécialité SES vous convient.
11,11.76,11.45,3.18,16.25,Des résultats moyens. Zyneb est une élève sérieuse qui doit participer davantage à l'oral et qui ne doit pas se décourager trop vite.


## Extract raw data for whole directory

In [14]:
directory = f'./Bulletins/{year}'
files = os.listdir(directory)

col_names = ['eleve', 'classe', "min", "max", "commentaire"]

def get_datastream_from_file(path):
  df = tabula.read_pdf(
      path,
      pages=1,
      stream=False,
      lattice=True,
      guess=True,
      )[0]

  if len(df.columns) != 7:
    return pd.DataFrame()

  df = df.drop([df.columns[0], df.columns[6]], axis=1)
  df = df.dropna()

  df.columns = col_names

  return df


files_subset = files
files_path = list(map(lambda name : directory + '/' + name, files_subset))

result = pd.DataFrame(columns=col_names)
i = 0 # Track files count
j = 0 # Track files actually processed
for file_path in files_path:
  i += 1 
  df = get_datastream_from_file(file_path)
  if not df.empty:
    j += 1
    print(f"processing file {i} of {len(files_path)}: {file_path}")
    result = result.append(df)
  else:
    print(f"dataframe empty: {file_path}")

print(f"processed {j} files of {len(files_path)}. Percentage processed: {j/len(files_path)*100}%")
result.to_csv(f'./out/{year}/data_raw.csv', index=None)

Got stderr: nov. 30, 2021 12:09:50 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
AVERTISSEMENT: New fonts found, font cache will be re-built
nov. 30, 2021 12:09:50 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
AVERTISSEMENT: Building on-disk font cache, this may take a while
nov. 30, 2021 12:09:52 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
AVERTISSEMENT: Finished building on-disk font cache, found 791 fonts



processing file 1 of 809: ./Bulletins/2020/CRESSIAUX Cybelia-Trimestre 1-1T2-N-1789.pdf
processing file 2 of 809: ./Bulletins/2020/GBAKA Amana Oceane-Trimestre 3-2NDE 7-N-3647.pdf
processing file 3 of 809: ./Bulletins/2020/TEIXEIRA Marina-Trimestre 1-2NDE11-N-6535.pdf
processing file 4 of 809: ./Bulletins/2020/ROYER Isaline-Trimestre 2-2NDE 4-N-2921.pdf
processing file 5 of 809: ./Bulletins/2020/ROGER Marilou-Trimestre 3-2NDE13-N-8441.pdf
processing file 6 of 809: ./Bulletins/2020/MARTINS Manola-Trimestre 1-1G7-N-1079.pdf
processing file 7 of 809: ./Bulletins/2020/DE ABREU Carla-Trimestre 2-2NDE 3-N-2743.pdf
processing file 8 of 809: ./Bulletins/2020/EXPERT Mano-Trimestre 2-2NDE 8-N-3793.pdf
processing file 9 of 809: ./Bulletins/2020/CHARTIER Loise-Trimestre 2-2NDE 3-N-2681.pdf
processing file 10 of 809: ./Bulletins/2020/MOKA Erica-Trimestre 1-1G4-N-247.pdf
processing file 11 of 809: ./Bulletins/2020/RABIER Lucas-Trimestre 2-2NDE 9-N-3985.pdf
processing file 12 of 809: ./Bulletins/2020

## Extract data from google docs

In [ ]:
df = pd.read_clipboard()
df.head()

,eleve,classe,min,max,commentaire
0,5.0,"9,06",0,19.5,Ensemble demeuré fragile.
1,8.0,"9,06",0,19.5,Ensemble fragile.
2,11.5,"9,06",0,19.5,Ensemble honorable.
3,3.5,"9,06",0,19.5,Ensemble demeuré fragile.
4,12.5,"9,06",0,19.5,Ensemble en recul mais qui demeure honorable.


In [ ]:
df.shape

(96, 5)

In [ ]:
df.to_csv('data_complete.csv', mode='a', header=None, index=None)

## Cleanup 

In [15]:
df = pd.read_csv(f'./out/{year}/data_raw.csv')
df.head(100)

,eleve,classe,min,max,commentaire
0,"18,29","12,12","5,13","18,29","Un anglais de qualité (tant à l'oral qu'à l'écrit). Cela n'excuse cependant pas votre manque d'attention et vos bavardages,\rtrop nombreux."
1,"18,80","12,82","6,40","18,80","Niveau d'anglais excellent. C'est très solide, au-delà même des attentes pour une classe de première. Vous avez les moyens\rde participer davantage."
2,"17,80","14,28","8,30","17,80","Un excellent début d'année, poursuivez ainsi !"
3,"15,00","14,51","11,00","19,50",Bon ensemble.
4,"10,60","10,87","5,47","16,71","Résultats corrects mais le comportement en classe n'est pas toujours approprié, il faut resté concentrée pour progresser."
...,...,...,...,...,...
95,"13,50","15,29","11,50","17,67",Une présence épisodique qui ne permet pas une appréciation globale. Dommage.
96,"11,50","13,98","11,00","17,50",Une bonne participation ce trimestre. Loïse démontre une attitude mature et responsable.
97,"17,56","11,28","3,50","18,50",Excellent trimestre. Loïse est une élève agréable et volontaire. Félicitations !
98,"14,67","11,92","6,40","17,75","Bon trimestre, le travail est toujours sérieux et régulier. J'attends maintenant une participation orale active."


In [16]:
df.to_csv(f'./out/{year}/data_clean.csv', index=None)

## Read

In [17]:
df = pd.read_csv(f'./out/{year}/data_clean.csv')

In [19]:
df['eleve'].head()

0    18,29
1    18,80
2    17,80
3    15,00
4    10,60
Name: eleve, dtype: object

In [20]:
df.shape

(9185, 5)

### Nan values

In [21]:
# Certains commentaires sont passés dans la colonne élève de la ligne suivante. On les repère avec un commentaire na et pas de nombre (mais du texte) dans la colonne eleve.
n = 0
for i in df[df['commentaire'].isna()].index:
  if re.search(r'\d|Abs|Disp|N.Rdu|N.Not', df['eleve'].iloc[i]) == None:
    if type(df['commentaire'].iloc[i-1]).__name__ == "str" and type(df['eleve'].iloc[i]).__name__ == "str":
      n += 1
      print(i)
      print(f"avant: {df['commentaire'].iloc[i-1]}")
      df['commentaire'].iloc[i-1] = df['commentaire'].iloc[i-1] + ' ' + df['eleve'].iloc[i]
      print(f"après: {df['commentaire'].iloc[i-1]}")
print(f"nombre de lignes modifiées: {n}")

9
avant: Le travail et le sérieux en classe sont trop inégaux. Il faut apprendre les leçons pour chaque cours et rester concentrée et
après: Le travail et le sérieux en classe sont trop inégaux. Il faut apprendre les leçons pour chaque cours et rester concentrée et calme pendant toute la durée du cours pour progresser.
12
avant: Très bon trimestre. De bons résultats car le travail est sérieux en classe. Restez bien concentrée et vous progresserez
après: Très bon trimestre. De bons résultats car le travail est sérieux en classe. Restez bien concentrée et vous progresserez encore.
19
avant: Excellent trimestre. L'état d'esprit est toujours positif et le travail sérieux. Très bonne participation orale. Pensez à bien
après: Excellent trimestre. L'état d'esprit est toujours positif et le travail sérieux. Très bonne participation orale. Pensez à bien approfondir les réponses écrites.
26
avant: Trimestre tout juste satisfaisant. Marina a des difficultés et travaille dur pour y palier. Il faut

In [22]:
# Drop na values from 'commentaire' column
# df = pd.read_csv('data_clean.csv')
df2 = df.dropna(axis="index", how="any", subset=['commentaire'])

In [23]:
# Other columns don't have any na value left
cols = ['eleve', 'classe','min', 'max']
[sum(df2[col].isna()) for col in cols]

[0, 0, 0, 0]

In [24]:
df2.isna().any()

eleve          False
classe         False
min            False
max            False
commentaire    False
dtype: bool

In [25]:
df.shape

(9185, 5)

In [26]:
df2.shape

(7931, 5)

In [27]:
df2.to_csv(f'./out/{year}/data_clean.csv', index=None)

### Grades cleanup

In [28]:
df = pd.read_csv(f'./out/{year}/data_clean.csv')
df.head()

,eleve,classe,min,max,commentaire
0,"18,29","12,12","5,13","18,29","Un anglais de qualité (tant à l'oral qu'à l'écrit). Cela n'excuse cependant pas votre manque d'attention et vos bavardages,\rtrop nombreux."
1,"18,80","12,82","6,40","18,80","Niveau d'anglais excellent. C'est très solide, au-delà même des attentes pour une classe de première. Vous avez les moyens\rde participer davantage."
2,"17,80","14,28","8,30","17,80","Un excellent début d'année, poursuivez ainsi !"
3,"15,00","14,51","11,00","19,50",Bon ensemble.
4,"10,60","10,87","5,47","16,71","Résultats corrects mais le comportement en classe n'est pas toujours approprié, il faut resté concentrée pour progresser."


#### Multiple grades

In [38]:
# How many rows with multiple grades
mask = df['eleve'].str.contains('\r') | df['classe'].str.contains('\r') 
mask.sum()

4

In [39]:
# Filter these
df = df[~mask]
df.to_csv(f'./out/{year}/data_clean.csv', index=False)

In [40]:
# Remaining non numerical values are semantic
df[~df['eleve'].str.contains('\d+[\.,]*\d*', case=False, regex=True)].value_counts(subset=['eleve'])

eleve
Disp     42
Abs      34
N.Not    33
Inap      7
dtype: int64

#### Formating numbers

In [41]:
cols = ['eleve', 'classe','min', 'max']
for col in cols:
    df[col] = df[col].str.replace(',','.', regex=True)

/var/folders/ff/2c14gfns0td1t7tmcw6mttp40000gn/T/ipykernel_7358/3933059145.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.replace(',','.', regex=True)


In [42]:
df = df.astype({"classe": 'float16', "min": 'float16', "max": 'float16'})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7751 entries, 0 to 7930
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   eleve        7751 non-null   object 
 1   classe       7751 non-null   float16
 2   min          7751 non-null   float16
 3   max          7751 non-null   float16
 4   commentaire  7751 non-null   object 
dtypes: float16(3), object(2)
memory usage: 227.1+ KB


In [43]:
df.shape

(7751, 5)

### Commentaires cleanup

In [44]:
# remove new line \r returns
mask = df['commentaire'].str.contains('\r')
len(df[mask])

1315

In [45]:
df['commentaire'] = df['commentaire'].str.replace('\r',' ', regex=True)

## Filter comments with explicit field name

In [46]:
# Trouver le nombre d'appréciations contenant un mot clé 
keywords = ["allema", 'anglais', "badmin", "basket", "blouse", "course", "dnl", "eps", "escalade", "espagn", "françai", "géog", "grec", "histoir", "italie",  "judo", "juridi", "littéra", "latin", "llce","math", "MPS", "musical", "musiq", "natation", "note", "philo", "phys", "russ", "scienc",  "séjou", "sport", "svt", "tp", "théât", "triche", "voyag", "yoga",]
found = {}
for keyword in keywords: 
    mask = df['commentaire'].str.contains(keyword, case=False, regex=False)
    found[keyword] = len(df.loc[mask, "commentaire"])

found

{'allema': 2,
 'anglais': 86,
 'badmin': 13,
 'basket': 7,
 'blouse': 2,
 'course': 4,
 'dnl': 13,
 'eps': 10,
 'escalade': 26,
 'espagn': 15,
 'françai': 40,
 'géog': 8,
 'grec': 2,
 'histoir': 9,
 'italie': 1,
 'judo': 14,
 'littéra': 4,
 'latin': 6,
 'llce': 3,
 'math': 55,
 'MPS': 55,
 'musical': 12,
 'musiq': 0,
 'natation': 7,
 'note': 142,
 'philo': 0,
 'phys': 13,
 'russ': 0,
 'scienc': 19,
 'séjou': 0,
 'sport': 7,
 'svt': 53,
 'tp': 14,
 'théât': 1,
 'triche': 1,
 'voyag': 2,
 'yoga': 4}

In [47]:
case_sensitive_keywords = ["SES", "s\.e\.s"]
case_sensitive_found = {}
for keyword in case_sensitive_keywords: 
    mask = df['commentaire'].str.contains(keyword, case=True, regex=True)
    case_sensitive_found[keyword] = len(df.loc[mask, "commentaire"])

case_sensitive_found

{'SES': 30, 's\\.e\\.s': 0}

In [48]:
# Total d'appréciations contenant un mot clé trop explicite
functools.reduce(lambda x,y: x+y, found.values(), 0) + functools.reduce(lambda x,y: x+y, case_sensitive_found.values(), 0)

680

In [49]:
# Filter ces appréciations 1/2
pattern = "|".join(keywords)
mask = df['commentaire'].str.contains(pattern, case=False, regex=True)
df = df.loc[~mask]

df.shape

(7144, 5)

In [50]:
# Filter ces appréciations 2/2
pattern = "|".join(case_sensitive_keywords)
mask = df['commentaire'].str.contains(pattern, case=True, regex=True)
df = df.loc[~mask]

df.shape

(7115, 5)

In [51]:
df.to_csv(f'./out/{year}/data_clean.csv', index=False)

In [52]:
df = pd.read_csv(f'./out/{year}/data_clean.csv')
df.head()

,eleve,classe,min,max,commentaire
0,17.80,14.28,8.30,17.80,"Un excellent début d'année, poursuivez ainsi !"
1,15.00,14.51,11.00,19.50,Bon ensemble.
2,10.60,10.87,5.47,16.70,"Résultats corrects mais le comportement en classe n'est pas toujours approprié, il faut resté concentrée pour progresser."
3,15.50,14.24,6.00,19.00,Bon trimestre. Continuez ainsi !
4,14.75,12.67,6.00,18.25,Un bon trimestre. Travail sérieux mais Cybélia doit essayer de participer davantage en classe;


## Filter stutdents names

In [53]:
# Create list of all student names from files
files = os.listdir(f'./Bulletins/{year}')
student_names = set()
for filename in files:
  name = re.search("^(?P<nom>[A-Z\- ]*) (?P<prenom>.*)-Trimestre", filename)
  if name:
    student_names.add(name.groups()[0])
    student_names.add(name.groups()[1])

student_names

{'AAZZOUZ',
 'ABICHOU',
 'ADDARKAOUI',
 'ALAMELOU',
 'ALHAMWI',
 'ALVES DA SILVA',
 'AMBOLET',
 'AMINE EL SAYED',
 'ASSALE',
 'AYMERIAL',
 'Abdel-Qhadouss',
 'Achille',
 'Adam',
 'Alice',
 'Alix',
 'Amana Oceane',
 'Amelie',
 'Ana-Maria',
 'Anaia',
 'Anas',
 'Antonin',
 'Asma',
 'Axelle',
 'Ayoub',
 'BAUDOIN',
 'BEAUDOT',
 'BEHARI',
 'BELGUASMI',
 'BENAKKAF',
 'BENAZERAF',
 'BENOIT',
 'BERTHAULT',
 'BIEUZEN',
 'BIHEMI',
 'BOGLER',
 'BOISSART',
 'BOISSAY',
 'BONIN-PELGAS',
 'BORDIER',
 'BOURDEAU',
 'BRINON',
 'BRISHOUAL',
 'BROSSARD',
 'BUCAMP',
 'Benjamin',
 'Bilel',
 'Bouchra',
 'Brewal',
 'CABRE',
 'CALVET',
 'CARRASCO',
 'CARRE',
 'CAUCHON',
 'CAUQUIS',
 'CHABERNAUD',
 'CHARTIER',
 'COLART',
 'COLLIAUT',
 'CONTASSOT',
 'COSPEREC',
 'COUADIER',
 'COURCOUX',
 'CRESSIAUX',
 'CUNHA',
 'CUVELIER',
 'Camille',
 'Carla',
 'Celia',
 'Chaima',
 'Charlotte',
 'Chloe',
 'Clara',
 'Clemence',
 'Cybele',
 'Cybelia',
 'DA SILVA',
 'DA SILVA PINTO',
 'DANCIU',
 'DE ABREU',
 'DEBAQUE',
 'DEHAINAULT

In [54]:
# Trouver le nombre d'appréciations contenant un nom d'élève (insensible aux accents)
student_names_found = {}
for name in student_names: 
    mask = df['commentaire'].apply(lambda x: unidecode(x)).str.contains(f" {unidecode(name)} ", case=False, regex=False)
    nb = len(df.loc[mask, "commentaire"])
    if nb > 0:
        student_names_found[name] = nb

In [55]:
# Bien vérifier la sortie pour détecter les anomalie (nom d'élève "LE" etc.)
student_names_found

{'Maelys': 10,
 'Carla': 4,
 'Raphael': 11,
 'Manon': 31,
 'Leopoldine': 10,
 'Ana-Maria': 6,
 'Lea': 13,
 'Sarah': 23,
 'Julien': 12,
 'Elicia': 7,
 'Jean': 9,
 'Iolene': 3,
 'Nicolas': 7,
 'Fariss': 11,
 'Paul': 11,
 'Nisa': 8,
 'Antonin': 10,
 'Mama': 5,
 'Matthieu': 5,
 'Marilou': 8,
 'Yaniss': 18,
 'Lewnaiesy': 7,
 'Payana': 10,
 'Brewal': 6,
 'Leni': 8,
 'Harmony': 3,
 'Selma-Nour': 4,
 'Alix': 6,
 'BENOIT': 1,
 'Chloe': 9,
 'Mabiza': 4,
 'Lilian': 4,
 'Victor': 3,
 'Laly': 4,
 'Pierrick': 3,
 'Heloise': 7,
 'Serine': 2,
 'Cybele': 8,
 'Neila': 11,
 'Ayoub': 9,
 'Samuel': 5,
 'Matteo': 1,
 'Marine': 20,
 'Isaline': 5,
 'Pauline': 27,
 'Siham': 7,
 'Erine': 8,
 'Lily': 8,
 'Juliette': 9,
 'Emma': 13,
 'Walid': 7,
 'Marie-Amelie': 10,
 'Viani': 13,
 'Romane': 11,
 'Nathan': 8,
 'Shahd': 6,
 'Rachel': 8,
 'Celia': 10,
 'Remy': 10,
 'Achille': 9,
 'Noor-Iman': 7,
 'Louna': 11,
 'Rony': 2,
 'Luna': 6,
 'Alice': 4,
 'Joudi': 13,
 'Rafaella': 13,
 'Axelle': 6,
 'Marvin': 11,
 'Flavie': 

In [58]:
# Retirer les anomalies
student_names_found.pop("LE")

542

In [56]:
# Total des noms d'élèves apparaissant en commentaire
functools.reduce(lambda summ, x: summ+x, student_names_found.values(), 0)

1324

In [57]:
# Fonction qui anonymise une string en cherchant un pattern de noms et les remplaçant par <name>, insensible aux accents et aux majuscules. 
# Le reste de la string est inchangé (reste accentué)
## Nous ne pouvons utiliser re.sub à cause des problèmes d'accents il faut enlever les accents avant de traiter et les **récupérer** par la suite.
def accent_i_replace(str, pattern):
    m = re.search(f"{unidecode(pattern)}", unidecode(str), re.IGNORECASE)
    
    # Search only returns the first occurence.
    while(m):
        accented = str[m.start():m.end()]
        str = str.replace(accented, '<name>')
        m = re.search(f"{unidecode(pattern)}", unidecode(str), re.IGNORECASE)
    else:
        return str

In [58]:
# Remplace les noms d'élèves (insensible aux accents) dans les appréciations par '<name>'
# Search pattern should be something like (^|\b)(ines)($|\b) to look for name at the begining and end of the comment, as well as between word boundaries (punctuation, spaces).
pattern = "|".join(map(lambda x: f"(^|\\b)({x})($|\\b)", student_names_found.keys()))
df['commentaire'] = df['commentaire'].apply(lambda x: accent_i_replace(x, pattern))

In [69]:
# Sample data to check everything is ok
df.sample(15)

,eleve,classe,min,max,commentaire
3942,15.70,13.10,8.75,19.20,"Ensemble très satisfaisant. Du travail, du sérieux. Bonne implication à l'oral."
2504,16.50,11.93,6.00,17.00,Excellent trimestre ; élève très sérieuse.
4135,14.90,13.92,10.40,17.90,Bon trimestre. Élève active en classe.
6649,6.63,13.90,6.63,19.81,"C'est un premier trimestre trop juste. Votre implication dans le travail est trop irrégulière, vous devez gagner en rigueur."
2838,9.85,9.14,5.00,13.54,Des résultats globalement corrects et en progrès. Elève sérieuse qui doit poursuivre ses efforts pour progresser. Courage !
5679,14.60,12.08,5.20,19.80,Bon trimestre. Zelhia est une élève impliquée.
4329,10.78,10.98,4.91,18.27,Des progrès durant ce trimestre. Continuez ainsi.
3083,11.50,11.50,7.00,16.00,Bilan passable.
2593,15.38,13.15,6.75,20.00,Très bon trimestre. Travail sérieux et bonne participation. Continue ainsi.
5957,10.79,13.29,8.79,19.05,"Ensemble juste dû à des difficultés importantes, vous devez homogénéiser vos efforts en classe et en dehors, pour progresser davantage. Cela passe par une accentuation du travail personnel et du travail en classe, en limitant également quelques bavardages."


In [79]:
# Manual cleanup 
pattern = "|".join(map(lambda x: f"(^|\\b)({x})($|\\b)", ["Zelhia" ,"Nisaest"]))
# df[df['commentaire'].str.contains("Nisa")]
df['commentaire'] = df['commentaire'].apply(lambda x: accent_i_replace(x, pattern))

In [80]:
df.to_csv(f'./out/{year}/data_clean.csv', index=False)

In [81]:
df = pd.read_csv(f'./out/{year}/data_clean.csv')

### Nettoyer les commmentaires tronqués

In [85]:
# Trouve les commentaires incluant des ponctuations mais ne finissant pas par une ponctuation et capture le bout de commentaire depuis la dernière ponctuation
# Pattern first group: capturing (greedy) group ending in punctuation (. ? !)
# Pattern second group : non capturing ignore group (everything else)
# If there is a match, .replace will only keep the 'keep' group. So it keeps one liner comments without punctuation
pattern = "(?P<keep>.*[\.\?\!])(?:.*[^\.\!\?]$)"
# df["commentaire"] = df["commentaire"].str.replace(pattern, lambda m: m.group("keep"))

# Check comments not ending by punctuation. Larger subset than pattern. Some of these are ok to keep.
pattern2 = "[^\.\!\?]$"

df.loc[df["commentaire"].str.contains(pattern2, regex=True), "commentaire"]


574                                                                                 Dispensé à l'année
712                                                                     Présentation orale perfectible
717                                                                         Dispensé pour l'évaluation
796                                                                  Bon trimestre quant aux résultats
905                                                                                            absente
                                                     ...                                              
6763    assez bon trimestre,bon trimestre en économie, entrez dans a logique juridique et ce sera bien
6767                                                         bon trimestre, continuez à vous impliquer
6769                                                                                     bon trimestre
6812                              bonne participation orale, le travail p

In [86]:
# Certains i ne sont pas bien passés
df["commentaire"] = df["commentaire"].str.replace("í", "i")

In [87]:
# Il y a quelques caractère russes dans un commentaire (trouvé par inspection des caractères uniques présents dans l'ensemble des commentaires)
list(df.loc[df['commentaire'].str.contains("и"), 'commentaire'])

["C'est un bon trimestre <name>. Vous êtes plus attentive, vous mûrissez, vous progressez. Continuez ainsi. Pour les friandises exotiques que vous avez partagees avec la classe : сп асибо!"]

In [88]:
# On filtre le commentaire russe
df = df.loc[~df['commentaire'].str.contains("и")]

## Inspect Disp, Abs, NNot comments

In [89]:
tags = ["Disp", "Abs", "N.Not", "Inap"]
df.loc[df['eleve'] == "Disp", "commentaire"]

61                                                                                                                                                Dispensée ce trimestre.
225                                                                                                                                                Dispensé ce trimestre.
285                                                                                                                                            Dispensée en fin de cycle.
395                                                                                                                                               Dispensée ce trimestre.
574                                                                                                                                                    Dispensé à l'année
717                                                                                                                                            Dispens

## Load

In [ ]:
df = pd.read_csv(f'./out/{year}/data_clean.csv')

## Save

In [90]:
df.to_csv(f'./out/{year}/data_clean.csv', index=False)

In [91]:
df.shape

(7114, 5)